In [1]:
import pynbody
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
from luminosity import surface_brightness

In [2]:
def bounding_box(snap):
    unit = snap['pos'].units
    for coord in 'x y z'.split():
        print("{}: {:10.2f}, {:10.2f} ({})".format(coord, snap[coord].min(), snap[coord].max(), unit))
    return [(float(snap[coord].min()), float(snap[coord].max())) for coord in 'x y z'.split()]

In [3]:
snap = "/home/michele/sim/MoRIA/M1-10_Verbeke2017/M10sim41001/snapshot_0036"
s = pynbody.load(snap)

In [4]:
max_boxsize = 4000

In [5]:
s.properties['boxsize'] = pynbody.units.Unit("{} kpc".format(max_boxsize))

In [6]:
width = 8
resolution=500

In [7]:
pynbody.analysis.halo.center(s.s)#, vel=False)
# pynbody.analysis.angmom.sideon(s.s)

In [8]:
half_light_radius = pynbody.analysis.luminosity.half_light_r(s, cylindrical=False)
half_light_radius

SimArray(1.24955014, 'kpc')

In [9]:
def ss_angmom(flux, r, v_los, v_disp):
    return np.sum(flux * r * np.abs(v_los)) / np.sum(flux * r * np.sqrt(v_los**2 + v_disp**2))
def compute_stellar_specific_angmom(snap, width, resolution, radii, eccentricity, a_delta, theta):
    luminosity_sim = surface_brightness(snap.s, band='v', width=width, resolution=resolution, mag_filter=None, cmap_name='Greys')
    v_los_sim = pynbody.plot.sph.image(snap.s, qty='vz', av_z='rho', width=width, resolution=resolution, log=False)
    v_disp_sim = pynbody.plot.sph.image(snap.s, qty='v_disp', av_z='rho', width=width, resolution=resolution, log=False)
    lum = to_astropy_quantity(luminosity_sim, units='mag/arcsec**2')
    v_los = to_astropy_quantity(v_los_sim)
    v_disp = to_astropy_quantity(v_disp_sim)
    lum_annuli = integrate_annulus(lum, center, radii, eccentricity, a_delta, theta)
    v_los_annuli = integrate_annulus(v_los, center, radii, eccentricity, a_delta, theta)
    v_disp_annuli = integrate_annulus(v_disp, center, radii, eccentricity, a_delta, theta)
    stellar_specific_angmom = ss_angmom(lum_annuli, radii, v_los_annuli, v_disp_annuli)
    return stellar_specific_angmom

In [10]:
max_r = 10 * half_light_radius
radius_slider = ipywidgets.IntSlider(min=0, max=max_r, value=5)
width_slider = ipywidgets.IntSlider(min=0, max=max_r, value=width)
vel_slider = ipywidgets.IntSlider(min=0, max=100, value=10, step=5)

def k(r, w, v):
    subsnap = s[pynbody.filt.Cuboid('{} kpc'.format(-r))]
    print(pynbody.analysis.angmom.ang_mom_vec(subsnap.s))
    pynbody.analysis.angmom.sideon(subsnap.s)
    print(pynbody.analysis.angmom.ang_mom_vec(subsnap.s))
#     fig, ((ax1, ax2, ax3), (cb1, cb2, cb3)) = plt.subplots(ncols=3, nrows=2, figsize=(12, 6))
    fig, ((ax1, ax2, ax3)) = plt.subplots(ncols=3, figsize=(12, 6))
    vlos = pynbody.plot.sph.image(subsnap.s, qty='vz', av_z=True, width=w, resolution=resolution, log=False,
                           vmin=-v, vmax=v, show_cbar=False, subplot=ax1)
    
#     fig, ax2 = plt.subplots()#surface_brightness(s, width=width, resolution=resolution);
    sigma = pynbody.plot.sph.image(subsnap.s, qty='v_disp', av_z=True, width=w, resolution=resolution, log=False,
                           show_cbar=False, subplot=ax2)

#     cbar2 = cb2.figure.colorbar(sigma);
#     cbar2.set_label('{} [{}]'.format("$\sigma", subsnap.s['v_disp'].units));

    flux = surface_brightness(subsnap, width=w, resolution=resolution, subplot=ax3, show_cbar=False);
#     cmap = plt.get_cmap(cmap_name)
#     cmap.set_bad('black')
#     cbar3 = cb3.figure.colorbar(flux);
#     cbar3.set_label('{} [mag/arcsec$^2$]'.format(band.upper()));
    
    fig.subplots_adjust(wspace=0.3)
    
w = ipywidgets.interactive(k, r=radius_slider, w=width_slider, v=vel_slider, show_cbar=False)
w

interactive(children=(IntSlider(value=5, description='r', max=12), IntSlider(value=8, description='w', max=12)…

In [11]:
max_r = 10 * half_light_radius
radius_slider = ipywidgets.IntSlider(min=0, max=max_r, value=5)
width_slider = ipywidgets.IntSlider(min=0, max=max_r, value=width)
vel_slider = ipywidgets.IntSlider(min=0, max=100, value=10, step=5)

def k(r, w, v):
    subsnap = s[pynbody.filt.Cuboid('{} kpc'.format(-r))]
#     print(pynbody.analysis.angmom.ang_mom_vec(subsnap.s))
    pynbody.analysis.angmom.sideon(subsnap.s)
#     print(pynbody.analysis.angmom.ang_mom_vec(subsnap.s))
    figsize=None#(6,6)
    fig1, ax1 = plt.subplots(figsize=figsize)
    vlos = pynbody.plot.sph.image(subsnap.s, qty='vz', av_z=True, width=w, resolution=resolution, log=False,
                           vmin=-v, vmax=v, show_cbar=True, subplot=ax1)
    
    fig2, ax2 = plt.subplots(figsize=figsize)
    sigma = pynbody.plot.sph.image(subsnap.s, qty='v_disp', av_z=True, width=w, resolution=resolution, log=False,
                           show_cbar=True, subplot=ax2)

#     cbar2 = cb2.figure.colorbar(sigma);
#     cbar2.set_label('{} [{}]'.format("$\sigma", subsnap.s['v_disp'].units));
    fig3, ax3 = plt.subplots(figsize=figsize)
    flux = surface_brightness(subsnap, width=w, resolution=resolution, subplot=ax3, show_cbar=True);
#     cmap = plt.get_cmap(cmap_name)
#     cmap.set_bad('black')
#     cbar3 = cb3.figure.colorbar(flux);
#     cbar3.set_label('{} [mag/arcsec$^2$]'.format(band.upper()));    
w = ipywidgets.interactive(k, r=radius_slider, w=width_slider, v=vel_slider, show_cbar=False)
w

interactive(children=(IntSlider(value=5, description='r', max=12), IntSlider(value=8, description='w', max=12)…